In [1]:
from pysnmp.entity.rfc3413.oneliner import cmdgen

cmd = cmdgen.CommandGenerator()

In [2]:
errorIndication, errorStatus, errorIndex, varBinds = cmd.getCmd(
    # v1
    # cmdgen.CommunityData('public', 0),
    # v2
    # cmdgen.CommunityData('public'),
    # v3
    cmdgen.UsmUserData(
        'demo', 'my_new_password', 'my_new_password', 
        cmdgen.usmHMACMD5AuthProtocol,
        cmdgen.usmDESPrivProtocol
    ),
    cmdgen.UdpTransportTarget(('ss.laisky.com', 161)),
    cmdgen.MibVariable('1.3.6.1.2.1.1.1.0')
)

print(
    'errorIndication: {}\n'
    'errorStatus: {}\n'
    'errorIndex: {}\n'
    'varBinds: {}'
    .format(errorIndication, errorStatus, errorIndex, varBinds)
)

errorIndication: No SNMP response received before timeout
errorStatus: 0
errorIndex: 0
varBinds: []


chexiang
===

- system = 1.3.6.1.2.1.1
- sysName = 1.3.6.1.2.1.1.5
- ifEntry = 1.3.6.1.2.1.2.2.1
- ifDescr = 1.3.6.1.2.1.2.2.1.2
- ifSpeed = 1.3.6.1.2.1.2.2.1.5

## 同步

In [88]:
import string
from pysnmp.smi import builder, view, error
from pysnmp.entity.rfc3413.oneliner import cmdgen


# 创建 cmd generator
cmdGen = cmdgen.CommandGenerator()  # 同步

mibBuilder = builder.MibBuilder()
mibBuilder.loadModules(
    'RFC-1213', 'RFC-3413'
)
mibView = view.MibViewController(mibBuilder)



errorIndication, errorStatus, errorIndex, varBinds = cmdGen.nextCmd(
    cmdgen.CommunityData('huawei@123'),
    cmdgen.UdpTransportTarget(('10.32.33.6', 161)),
    cmdgen.MibVariable('IF-MIB', 'ifOperStatus')
)


print(errorIndication)
print(errorStatus)
print(errorIndex)
print('*' * 50)

for row in varBinds:
    for oid, val in row:
        print(oid.getMibSymbol()[-1][0], val.prettyPrint())

None
0
0
**************************************************
1 'up'
2 'up'
3 'down'
5 'up'
6 'up'
7 'up'
8 'up'
9 'up'
10 'up'
11 'up'
12 'up'
13 'up'
14 'up'
15 'up'
16 'up'
17 'up'
18 'up'
19 'up'
20 'up'
21 'up'
22 'up'
23 'up'
24 'up'
25 'up'
26 'up'
27 'up'
28 'up'
29 'up'
30 'up'
31 'up'
32 'up'
33 'up'
34 'up'
35 'down'
36 'down'
37 'down'
38 'up'
39 'up'
40 'up'
41 'up'
42 'up'
43 'up'
44 'up'
45 'up'
46 'up'
47 'up'
48 'up'
49 'up'
50 'up'
51 'down'
52 'up'
53 'up'
54 'down'
55 'down'
56 'down'
57 'down'
58 'down'
59 'up'
60 'up'


## 异步

In [77]:
import string
from pysnmp.smi import builder, view, error
from pysnmp.entity.rfc3413.oneliner import cmdgen


# 创建 cmd generator
cmdGen = cmdgen.CommandGenerator()  # 同步
cmdGen = cmdgen.AsynCommandGenerator()  # 异步

mibBuilder = builder.MibBuilder()
mibBuilder.loadModules(
    'RFC-1213', 'RFC-3413'
)
mibView = view.MibViewController(mibBuilder)


mib_vars = (
    {'desc': 'ifOperStatus', 'mib': cmdgen.MibVariable('IF-MIB', 'ifOperStatus')},
    {'desc': 'ifSpeed', 'mib': cmdgen.MibVariable('IF-MIB', 'ifSpeed')},
    {'desc': 'ifInErrors', 'mib': cmdgen.MibVariable('IF-MIB', 'ifInErrors')},
)


def callback(sendRequestHandle, errorIndication, errorStatus, errorIndex, varBinds, cbCtx):
    
    for row in varBinds:
        for oid, val in row:
            if cbCtx['desc'] not in oid.prettyPrint():
                return 0
            
            print(oid.prettyPrint(), val.prettyPrint())
            
    return 1  # 告诉 walk 继续运行
            


for mib_var in mib_vars:
    cmdGen.nextCmd(
        cmdgen.CommunityData('huawei@123'),
        cmdgen.UdpTransportTarget(('10.32.33.6', 161)),
        (mib_var['mib'],),
        (callback, mib_var),
        True, True
    )

    
# 启动
cmdGen.snmpEngine.transportDispatcher.runDispatcher()

IF-MIB::ifOperStatus."1" 'up'
IF-MIB::ifSpeed."1" 0
IF-MIB::ifInErrors."1" 0
IF-MIB::ifOperStatus."2" 'up'
IF-MIB::ifSpeed."2" 0
IF-MIB::ifInErrors."2" 0
IF-MIB::ifOperStatus."3" 'down'
IF-MIB::ifSpeed."3" 0
IF-MIB::ifInErrors."3" 0
IF-MIB::ifOperStatus."5" 'up'
IF-MIB::ifSpeed."5" 100000000
IF-MIB::ifInErrors."5" 2
IF-MIB::ifOperStatus."6" 'up'
IF-MIB::ifSpeed."6" 100000000
IF-MIB::ifInErrors."6" 0
IF-MIB::ifOperStatus."7" 'up'
IF-MIB::ifSpeed."7" 100000000
IF-MIB::ifInErrors."7" 0
IF-MIB::ifOperStatus."8" 'up'
IF-MIB::ifSpeed."8" 100000000
IF-MIB::ifInErrors."8" 0
IF-MIB::ifOperStatus."9" 'up'
IF-MIB::ifSpeed."9" 100000000
IF-MIB::ifInErrors."9" 0
IF-MIB::ifOperStatus."10" 'up'
IF-MIB::ifSpeed."10" 100000000
IF-MIB::ifInErrors."10" 4
IF-MIB::ifOperStatus."11" 'up'
IF-MIB::ifSpeed."11" 100000000
IF-MIB::ifInErrors."11" 2
IF-MIB::ifOperStatus."12" 'up'
IF-MIB::ifSpeed."12" 100000000
IF-MIB::ifInErrors."12" 0
IF-MIB::ifOperStatus."13" 'up'
IF-MIB::ifSpeed."13" 100000000
IF-MIB::ifInErro

In [69]:
cmdGen.nextCmd??